In [1]:
'''
简化版:
    1. 在数据集上遍历每一个alpha，
    2. 将剩下的alpha集合中随机选择另一个alpha, 从而构建alpha对
    我们要同时改变两个alpha，因为有一个约束条件 ∑a(i).label(i) = 0
    改变一个会可能会导致约束失效

    构建一个辅助函数，在某个区间内随机选一个整数，
    构建另一个辅助函数，在数值太大时对其进行调整
'''
from numpy import *
from time import sleep


def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat


def selectJrand(i, m):
    """
    只要函数值数等于输入值i，函数就会进行随机选择
    :param i: alpha的下标
    :param m: 所有alpha的数目
    :return:
    """
    j = i  # we want to select any J not equal to i
    while (j == i):
        j = int(random.uniform(0, m))
    return j


def clipAlpha(aj, H, L):
    """
    用于调整大于H 或小于 L 的alpha值
    :param aj:
    :param H:
    :param L:
    :return:
    """
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj


def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    """
    创建一个alpha向量并将其初始化为0向量
    当迭代次数小于最大迭代次数时(外循环):
        对数据集中的每个数据向量(内循环):
            如果该数据向量可以被优化:
                随机选择另一个数据向量
                同时优化这两个向量
                如果这两个向量不能被优化，退出内循环

    :param dataMatIn: 数据集
    :param classLabels: 类别标签
    :param C: 常数C 0.6
    :param toler: 容错率 0.001
    :param maxIter: 退出当前最大的循环次数 40
    :return:

    构建函数时采用了通用的接口，这样就可以对算法和数据源进行组合和配对处理
    """
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()
    b = 0
    m, n = shape(dataMatrix)
    alphas = mat(zeros((m, 1)))
    iter = 0  # 没有任何alpha改变的情况下的便利数据集的次数

    while (iter < maxIter):
        alphaPairsChanged = 0  # 记录alpha时候已经进行优化
        # 如果alpha可以更改进入优化过程 m: 100
        for i in range(m):
            # fXi 是我们预测的类别
            fXi = float(multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T)) + b
            # 基于fXi 和真实结果的对比，就可以计算误差Ei
            Ei = fXi - float(labelMat[i])  # if checks if an example violates KKT conditions
            # 如果误差很大，就可以对该数据实例说对应的alpha值进行优化, 不管是正间隔还是负间隔,都会被测试优化 and alpha can't equal 0 | C, 如果已经在边界上，就不值得优化了
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                # 随机选择第二个alpha
                j = selectJrand(i, m)
                fXj = float(multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])
                # python采用引用的方式传递所有列表,所以必须分配新内存
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()

                # 保证alpha在 0 和　C　之间
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H:
                    print("L==H")
                    continue
                # eta是alpha[j]的最优修改量
                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T - dataMatrix[i, :] * dataMatrix[i, :].T - dataMatrix[
                                                                                                            j,
                                                                                                            :] * dataMatrix[
                                                                                                                 j, :].T

                if eta >= 0:
                    print("eta>=0")
                    continue
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough")
                    continue

                # 对i进行修改，修改量与j相同，但方向相反
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])  # update i by the same amount as j
                # the update is in the oppostie direction
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[i, :].T - labelMat[
                    j] * (alphas[j] - alphaJold) * dataMatrix[i, :] * dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[j, :].T - labelMat[
                    j] * (alphas[j] - alphaJold) * dataMatrix[j, :] * dataMatrix[j, :].T
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2) / 2.0
                alphaPairsChanged += 1
                print("iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))

        if (alphaPairsChanged == 0):
            iter += 1
        else:
            iter = 0
        print("iteration number: %d" % iter)
    return b, alphas


In [2]:
class optStruct():
    def __init__(self, dataMatIn, classLables, C, toler):
        self.x = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2))) # eCache的第一列给出的是eCache是否有效的标志位
    
def calcEk(oS, k):
    # 对于给定的alpha,能计算E并返回
    fXk = float(multiply(oS.alphas, oS.labelMat).T*(oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJ(i, oS, Ei):
    # 用于选择第二个alpha或者说内循环的alpha值,alpha值Ei和下标i有关,
    """
    输入值Ei, 在缓存中是有效的,
    
    """
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei]
    validEcacheList = nonzero(oS.eCache[:,0].A)[0] # 构建一个非零表, 以输入列表为目录的列表值
    # nonzero语句返回的是非零E值所对应的alpha值,而不是E值本身.程序会在所有的值上进行循环并选择其中使得改变最大的那个值
    # 如果第一次loop, 随机选择一个alpha
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:
            continue
        Ek = calcEk(oS, k)
        deltaE = abs(Ei - Ek)
        if (deltaE > maxDeltaE):
            maxK = k
            maxDeltaE = deltaE
            Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):
    # 计算误差值并存入缓存中,在对alpha值进行优化之后需要这个值
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1, Ek]    

In [ ]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.c)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alpha[i] > 0)):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[j])
            H = min(oS.C, oS.C, oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H:
            print("L == H")
            return 0
        eta = 2.0 * oS.X[i, :]*oS.X[j, :].T - oS.X[i, :]*oS.X[i, :]*oS[i,:].T - oS.X[j, :]*oS.X[j,:].T
        